In [1]:
from utils.schedule_utils import load_data_split,table_to_str,table_to_str_sql,find_intersection_and_add_row_id,Prepare_Data_for_Operator_Sequence,format_document,batch_rerank_scores,ROLLBACK,merge_clean_and_format_df_dict,retrieve_rows_by_subtables,process_error_analysis_list
import pickle
import numpy as np
import pandas as pd
import os
import json
from FlagEmbedding import FlagReranker
from tqdm import tqdm
from typing import Dict, Any
from utils.evaluator import Evaluator
from utils.prompt_generate import build_wikitq_prompt_from_df,evaluate_predictions,filter_dataframe_from_responses,fix_sql_query,match_subtables,retrieve_rows_by_subtables,build_tab_fact_prompt_from_df
from utils.async_llm import infer_prompts

/home/aizoo/miniconda3/envs/hstar/lib/python3.9/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


In [2]:
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer
import pandas as pd
import numpy as np
from time import sleep
import argparse
from vllm.lora.request import LoRARequest
import json
from tqdm import tqdm
import os 

INFO 10-22 00:27:04 [__init__.py:216] Automatically detected platform cuda.


In [33]:
## 参数记录 for tab_fact
N_PARALLEL = 32 ## only for convert_df_type_parallel.py of data preprocess，一次性离线的
embedding_base_model_path = '/data/workspace/yanmy/models/bge-m3'
# llm_path = '/data/workspace/yanmy/models/Qwen3-30B-A3B-Instruct-2507-FP8'
llm_path = '/data/workspace/yanmy/models/Qwen2.5-7B-Instruct'
# llm_path = '/data/workspace/yanmy/models/Qwen3-4B-Instruct-2507'
router_model_path = '/data/workspace/yanmy/HybridRAG/H-STAR/router/bge-m3-router-tab_fact-hn' ## fine-tuned router model path
check_moedl_path = '/data/workspace/yanmy/HybridRAG/H-STAR/check/output/bge-reranker-v2-m3-finetuned-tab_fact' ## check model path
tmp_save_path = 'datasets/schedule_test/tab_fact_api_30B' ## 临时存储路径
dataset_name = 'tab_fact'
split = 'test_small'
tau = 0.75
check_tau = 0.8
ALL_LABELS = [
    'Base', 'Select_Row', 'Select_Column', 'Execute_SQL', 'RAG_20_5', 
]


### Pipeline 的过程分为以下调度步骤
- 所有调度需要的文件均放在了 `schedule_test/datasets/wikitq`与`schedule_test/datasets/tab_fact`文件夹里面
1. （独立的）跑下面的指令：`python convert_df_type_parallel.py --dataset_name tab_fact --split test_small --output_path datasets/schedule_test/tab_fact/wikitq_df_processed.npy --num_workers 32`
2. (独立的)读取 `datasets/schedule_test/tab_fact/wikitq_df_processed.npy`,构造 inference_router 的文件，输出到`datasets/schedule_test/tab_fact/router_query.pkl`
3. (独立的)构造 DB
4. (GPU-21GB) inference router model，输出route result 到 `datasets/schedule_test/tab_fact/inference_result.pkl` 通过命令：`python inference_router.py --input_path datasets/schedule_test/tab_fact/router_query.pkl --model_path /data/workspace/yanmy/HybridRAG/H-STAR/router/bge-m3-router-tab_fact-hn --output_path datasets/schedule_test/tab_fact/inference_result.pkl`
5. (CPU) 解析 router 结果，生成初步的路径并开始任务调度(重排)
6. (GPU-1.8G) 根据 `datasets/schedule_test/tab_fact/RAG_index.npy`，执行 RAG 任务，根据下面的指令
7. (GPU-vLLM) LLM 根据初步路径，对`Select_Row`, `Select_Column`, `Execute_SQL`执行中间命令
8. (CPU) 根据中间结果，link db 做sql command parse，执行 sql,返回 subtable_list
9. (GPU-1.8G) 加载并运行 Check Model，跑 3 次（FORWARD 机制）
10. (GPU-vLLM) add execute SQL
11. (GPU-vLLM) generate qa command, inference vLLM, evaluate


In [ ]:
## step:1
f'python convert_df_type_parallel.py --dataset_name {dataset_name} --split {split} --output_path {tmp_save_path}/wikitq_df_processed.npy --num_workers {N_PARALLEL}'

In [4]:
### 读取必要文件（step 2）
dataset = load_data_split(dataset_name,split)
wikitq_df_processed = np.load(f'{tmp_save_path}/wikitq_df_processed.npy',allow_pickle=True).item()


In [10]:
## step 2: 构建semantic router的输入文件，并输出
semantic_router = {}
# semantic_router[
for index in range(len(dataset)):
    semantic_router[index] = {}
    semantic_router[index]['query'] = dataset[index]['question']
    semantic_router[index]['title'] = dataset[index]['table']['page_title']
    semantic_router[index]['table'] = wikitq_df_processed[index]
    label_list = []
    semantic_router[index]['label'] = label_list
with open(f'{tmp_save_path}/router_query.pkl','wb') as f:
    pickle.dump(semantic_router,f)

In [6]:
## step 3: Consturct DB
from utils.multi_db_v2 import NeuralDB, Executor
table_titles = [dataset[i]['table']['page_title'] for i in range(len(dataset))]
db = NeuralDB(tables=wikitq_df_processed, table_titles=table_titles)
executor = Executor()

Initializing NeuralDB...


Creating tables in DB:   0%|          | 0/2024 [00:00<?, ?it/s]

In [9]:
## step 4: inference router model
f'python inference_router.py --input_path {tmp_save_path}/router_query.pkl --model_path {router_model_path} --output_path {tmp_save_path}/inference_result.pkl'

'python inference_router.py --input_path datasets/schedule_test/tab_fact_api_30B/router_query.pkl --model_path /data/workspace/yanmy/HybridRAG/H-STAR/router/bge-m3-router-tab_fact-hn --output_path datasets/schedule_test/tab_fact_api_30B/inference_result.pkl'

In [11]:
## step 5
with open(f'{tmp_save_path}/inference_result.pkl', 'rb') as f:
    error_analysis_row = pickle.load(f)
ranked_result = process_error_analysis_list(error_analysis_row, truncate=True, tau=0.75)


In [34]:
## Organize LLM query list
LLM_query_list = {}
for method in ALL_LABELS:
    LLM_query_list[method] = {}
    LLM_query_list[method]['index'] = []
    LLM_query_list[method]['query'] = []
    LLM_query_list[method]['qa'] = []
for index in range(len(dataset)):
    for method in ALL_LABELS:
        if method in ranked_result[index]:
            # method_list = ranked_result[index][method]
            # method_list.append(index)
            LLM_query_list[method]['index'].extend([index])
            if method=='Select_Column':
                prompt = build_wikitq_prompt_from_df(dataset,wikitq_df_processed[index],index,template_path='../prompts/col_select_sql.txt',processed=True)
                LLM_query_list[method]['query'].extend([prompt])
            elif method=='Select_Row':
                prompt = build_wikitq_prompt_from_df(dataset,wikitq_df_processed[index],index,template_path='../prompts/row_select_sql.txt',processed=True)
                LLM_query_list[method]['query'].extend([prompt])
            elif method == 'Execute_SQL':
                prompt = build_wikitq_prompt_from_df(dataset,wikitq_df_processed[index],index,template_path='../prompts/sql_reason_tabfact.txt',processed=True)
                LLM_query_list[method]['query'].extend([prompt])
            # elif method == 'Base':
            #     prompt = build_wikitq_prompt_from_df(dataset,wikitq_df_processed[index],index,template_path='../prompts/text_reason_tabfact.txt',processed=True)
            #     LLM_query_list[method]['qa'].extend([prompt])


In [13]:
## 暂存 RAG 的 list，输出到路径
np.save(f'{tmp_save_path}/RAG_index.npy',LLM_query_list['RAG_20_5']['index'])

- 执行`Hybrid_Retrieve.py` 任务

In [14]:
## step 6: RAG
f"python Hybrid_Retrieve_Update_dict.py \
    --model_path {embedding_base_model_path} \
    --dataset_name {dataset_name} \
    --split {split} \
    --index_path {tmp_save_path}/RAG_index.npy \
    --output_path {tmp_save_path}/Hybrid_Retrieve_output.npy \
    --max_rows 50 \
    --max_cols 10 \
    --processed_df_path {tmp_save_path}/wikitq_df_processed.npy \
    --rewrite_query_path datasets/schedule_test/{dataset_name}/rewrite_query.npy"

'python Hybrid_Retrieve_Update_dict.py     --model_path /data/workspace/yanmy/models/bge-m3     --dataset_name tab_fact     --split test_small     --index_path datasets/schedule_test/tab_fact_api_30B/RAG_index.npy     --output_path datasets/schedule_test/tab_fact_api_30B/Hybrid_Retrieve_output.npy     --max_rows 50     --max_cols 10     --processed_df_path datasets/schedule_test/tab_fact_api_30B/wikitq_df_processed.npy     --rewrite_query_path datasets/schedule_test/tab_fact/rewrite_query.npy'

In [35]:
## load Hybrid_Retrieve 的结果
RAG_20_5 = np.load(f'{tmp_save_path}/Hybrid_Retrieve_output.npy',allow_pickle=True).item()

In [52]:
## step 7: api vLLM
### Query LLM with async engine，启动模型采用 sh llm_server.sh，这个是不是可以放在调度的第一个环节，不计入时间

metrics_rows = {}
summary = {}
for method in ['Select_Row','Select_Column']:
    prompt_list = LLM_query_list[method]['query']
    response_list,metrics_rows[method],summary[method] = infer_prompts(prompt_list, temperature=0.7, top_p=0.8, sample_num=2, llm_path = llm_path)
    LLM_query_list[method]['response'] = response_list
for method in ['Execute_SQL']:
    prompt_list = LLM_query_list[method]['query']
    response_list,metrics_rows[method],summary[method] = infer_prompts(prompt_list, temperature=0.7, top_p=0.8, sample_num=3, llm_path = llm_path)
    LLM_query_list[method]['response'] = response_list

Processed prompts:   0%|          | 0/18 [00:00<?, ?it/s]

Processed prompts: 100%|██████████| 322/322 [00:58<00:00,  5.48it/s, est. speed input=11959.07 toks/s, output=2546.65 toks/s]


- row_sql

In [37]:
# row_sql = pd.read_csv(f'../datasets/pipeline/{dataset_name}_{split}_row_sql_output_4B.csv',index_col=0)
from tqdm.notebook import tqdm
sub_table_list_all = {}
filtered_tables_row = {}
row_sql_index_list = LLM_query_list['Select_Row']['index']
row_sql_response_list = LLM_query_list['Select_Row']['response']
for i in range(len(row_sql_index_list)):
    sample_num = [0,1]
    sub_table_list = []
    for sample_index in sample_num:
        ## 相对距离
        index = row_sql_index_list[i]
        #  = row_sql_response_list[i][sample_index]
        # relevant_index = row_sql_index_list.index(index)
        original_text = row_sql_response_list[i][sample_index]
        sql = fix_sql_query(response_text=original_text,table_df=wikitq_df_processed[index],table_title=table_titles[index])
        try:
            result_1 = executor.sql_exec(sql.replace('``','`').replace("COUNT(*)", "*"), db, table_id=index)
            sub_table_list.append(pd.DataFrame(result_1['rows'],columns=result_1['header']))
        except:
            continue
    print(dataset[index]['question'],dataset[index]['answer_text'])
    # match_subtables(large_table=wikitq_df_processed[index], sub_tables=[pd.DataFrame(result_1['rows'],columns=result_1['header'])])
    # pd.DataFrame(result_1['rows'],columns=result_1['header'])
    sub_table_list_all[index] = sub_table_list
    filtered_df = retrieve_rows_by_subtables(wikitq_df_processed[index],sub_table_list)
    if len(filtered_df) == 0:
        filtered_df = wikitq_df_processed[index]
    filtered_tables_row[index] = filtered_df

Executing on table_id 2 ('w2'): SELECT a FROM cgame c when cdate c when cjana a 3;
Executing on table_id 2 ('w2'): 
the wildcats lost one game in september and two games in november 1
Executing on table_id 3 ('w3'): 
Executing on table_id 3 ('w3'): 
the wildcats played two games in september , four games in october , and four games in november 1
Executing on table_id 7 ('w7'): 
Executing on table_id 7 ('w7'): 
the wildcats loss one game in november and two games in september 0
Executing on table_id 8 ('w8'): 
Executing on table_id 8 ('w8'): 
the wildcats played four games in september , two games in october , and two games in november 0
Executing on table_id 491 ('w491'): 
Executing on table_id 491 ('w491'): SELECT the SQL query to achieve this task:   CREATE TABLE IF NOT EXISTS players (  (  columns:  id INTEGER PRIMARY KEY,  AUTOINCREMENT,   name name full last 1 score VARCHAR   inassistant;
north avenue plain street market street has both 12.2 and 12.8 listed in the milepost column 

- col_sql

In [38]:
filtered_tables = {}
filtered_headers = {}
col_sql_index_list = LLM_query_list['Select_Column']['index']
col_sql_response_list = LLM_query_list['Select_Column']['response']
# for ind in tqdm(range(len(wikitq_df_processed))):
for i in range(len(col_sql_index_list)):
    ind = col_sql_index_list[i]
    
    input_df = wikitq_df_processed[ind]
    response_list = col_sql_response_list[i]
    assert isinstance(response_list,list)
    filtered_table, final_headers = filter_dataframe_from_responses(response_list, input_df, add_row_id=True)
    filtered_tables[ind] = filtered_table
    filtered_headers[ind] = final_headers

- SQL-exec

In [39]:
from tqdm.notebook import tqdm
sample_num = 3
sql_exec_df = {}
valid_parse = 0
warning_list = []
sql_executable_count = []
exec_sql_index_list = LLM_query_list['Execute_SQL']['index']
exec_sql_response_list = LLM_query_list['Execute_SQL']['response']
# for index in tqdm(range(len(wikitq_df_processed))):
for i in range(len(exec_sql_index_list)):
    index = exec_sql_index_list[i]
    sql_exec_df[index] = []
    for sample_ind in range(sample_num):
        
        # original_text = eval(wikitq_df_output.iloc[index,-1])[sample_ind]
        original_text = exec_sql_response_list[i][sample_ind]
        sql = fix_sql_query(response_text=original_text,table_df=wikitq_df_processed[index],table_title=table_titles[index])
        if [index,sample_ind] in warning_list:
            sql = ''
            print(index,sample_ind)
        if sql!='':
            # print(result)
            try:
                result = executor.sql_exec(sql.replace('``','`'), db, table_id=index,add_row_id = True)
                df = pd.DataFrame(result['rows'],columns=result['header'])
            except:
                df = pd.DataFrame()
        else:
            df = pd.DataFrame()
        sql_exec_df[index].append(df)
        if(len(df)>0):
            valid_parse+=1
            sql_pair = {}
            sql_pair['id'] = index
            sql_pair['sample_ind'] = sample_ind
            sql_pair['sql'] = sql
            sql_pair['table'] = df
            sql_executable_count.append(sql_pair)
sql_exec_df_output = merge_clean_and_format_df_dict(sql_exec_df)
    # print(original_text)

Executing on table_id 0 ('w0'): SELECT opponent, ` FROM w WHERE game BETWEEN 1 AND 4;
Executing on table_id 0 ('w0'): SELECT opponent, record FROM w WHERE result[result] == 'loss' LIMIT 4;
Executing on table_id 2 ('w2'): plaintext 1. Nature of the Question   - This question involves identifying specific games outcomes (losses) for the Wildcats in particular months (September and November).   - This is a task suited for SQL. 2 Final Output   - Step 1: Filter the records where the date result 'loss' for both September and November     SELECT ` FROM w WHERE rresult = 'loss ' AND AND r date October OR AND strftime('%m', rdate ) ' == '0 ' OR AND   - Step  i: Filter the rows again have 'loss ' for November     SELECT row FROM wWHERE rresult '== loss ' AND strftime('%m', rdate ' == ' 11 '   - Combining the steps into one query     SELECT * FROM wWHERE rresult 'loss ' ANDND (strftime('%m ', rdate ') == '  in ' OROR strftime('%m ',rdate ') == '  11 ';
Executing on table_id 2 ('w2'): SELECT * FR

/data/workspace/yanmy/HybridRAG/H-STAR/utils/schedule_utils.py:294: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  final_df = unique_df.fillna('')
/data/workspace/yanmy/HybridRAG/H-STAR/utils/schedule_utils.py:294: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  final_df = unique_df.fillna('')
/data/workspace/yanmy/HybridRAG/H-STAR/utils/schedule_utils.py:294: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future beha

In [40]:
processed_table = {} ## processed_table包括了inference LLM后所有结果的聚合,key 与 method_key 保持一致
processed_table['Base'] = wikitq_df_processed
processed_table['Select_Row'] = filtered_tables_row
processed_table['Select_Column'] = filtered_tables
processed_table['RAG_20_5'] = RAG_20_5
processed_table['Execute_SQL'] = sql_exec_df_output ## SQL聚合结果！
processed_table['Execute_SQL_count'] = sql_executable_count ## SQL单个结果，用作 CrossEncoder 跨模态查询

In [41]:
np.save(f'{tmp_save_path}/processed_table.npy',processed_table)

- Run Check Model

In [42]:
# 对每一条数据，维护一个 Sequence_Operator
# 每条数据应该对应一个 Dict,这个Dict 包含了3 个参数：
    # id: index
    # Sequence: Operator Sequence
    # Terminated: True/False
    # Check Score: CrossEncoder 分数
    # Check Status: True/False,当前是否被计算
# 对每条数据，如果 Terminated = True，则不参与计算
# 对应的，如果 Terminated = False，那么查看 Check Status,如果 Statue=False，那么计算 Check_Score
# 每一轮计算之后，Check_Score 更新，Check Statue更新，该轮开始判定，用一个 threshold
# 如果 Check Status = True, Check Score >= Threshold，那么标注 Terminated = True，当前路径被接收
# 如果 Check Status = True, Check Score < Threshold，那么Terminated = False, Check Status = False, Check Score 清空
# Check Score 触发 RollBack 机制，即除了‘Execute_SQL'外，Extract 回退，重新计算当前流程
# 如果当前操作符只有'Execute_SQL’和空（所有 RollBack 均未被接收），那么触发 FORWARD,Terminated  = True，强制修改 Operator Sequence 为['Execute_SQL']
Check_Model_Data_Sequence = {}
for key in ranked_result.keys(): ## 遍历所有数据,初始化
    start_sequence = ranked_result[key]
    Check_Model_Data_Sequence[key] = {}
    Check_Model_Data_Sequence[key]['id'] = key
    Check_Model_Data_Sequence[key]['Sequence'] = start_sequence
    if start_sequence == ['Base'] or start_sequence == ['Execute_SQL']: ## Terminated 不参与计算
        Check_Model_Data_Sequence[key]['Terminated'] = True
        Check_Model_Data_Sequence[key]['Check_Status'] = False
        Check_Model_Data_Sequence[key]['Check_Score'] = 0.0
    else:
        Check_Model_Data_Sequence[key]['Terminated'] = False
        Check_Model_Data_Sequence[key]['Check_Status'] = False
        Check_Model_Data_Sequence[key]['Check_Score'] = 0.0
# Check_Model_Data_Sequence[4]
for key in Check_Model_Data_Sequence.keys():
    data_entry = Prepare_Data_for_Operator_Sequence(key,Check_Model_Data_Sequence[key]['Sequence'],dataset, processed_table)
    Check_Model_Data_Sequence[key]['data_entry'] = data_entry
## Check
    

- load ReRanking Model

In [43]:
reranker_model = FlagReranker(check_moedl_path, use_fp16=True)

In [44]:
check_tau = 0.9
for loop in range(3):
    updated_data = batch_rerank_scores(reranker_model, Check_Model_Data_Sequence, batch_size=4)
    Check_Model_Data_Sequence = updated_data
    ## Check Terminal Status
    for key in Check_Model_Data_Sequence.keys():
        if Check_Model_Data_Sequence[key]['Terminated'] == True:
            continue
        else:
            if Check_Model_Data_Sequence[key]['Check_Status'] == True:
                if Check_Model_Data_Sequence[key]['Check_Score'] >= check_tau: ## 0.5 threshold
                    Check_Model_Data_Sequence[key]['Terminated'] = True
                else:
                    Check_Model_Data_Sequence[key]['Terminated'] = False
                    Check_Model_Data_Sequence[key]['Check_Status'] = False
                    Check_Model_Data_Sequence[key]['Check_Score'] = 0.0
                    current_sequence = Check_Model_Data_Sequence[key]['Sequence']
                    ROLLBACK_seq, terminated_flag = ROLLBACK(current_sequence)
                    Check_Model_Data_Sequence[key]['Sequence'] = ROLLBACK_seq
                    Check_Model_Data_Sequence[key]['Terminated'] = terminated_flag
                    if terminated_flag == False:
                        data_entry = Prepare_Data_for_Operator_Sequence(key,ROLLBACK_seq,dataset, processed_table)
                        Check_Model_Data_Sequence[key]['data_entry'] = data_entry
            else:
                raise ValueError("Check Status should be True when Terminated is False after reranking.")

Filtering items to rerank (Terminated=False and data_entry exists)...
Computing scores for 390 items with batch size 4...


initial target device:   0%|          | 0/2 [00:00<?, ?it/s]/home/aizoo/miniconda3/envs/hstar/lib/python3.9/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
initial target device:  50%|█████     | 1/2 [00:08<00:08,  8.91s/it]/home/aizoo/miniconda3/envs/hstar/lib/python3.9/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
Chunks: 100%|██████████| 2/2 [00:04<00:00,  2.21s/it]


Updating scores in the original dictionary...
Reranking complete.
Filtering items to rerank (Terminated=False and data_entry exists)...
Computing scores for 295 items with batch size 4...


Chunks: 100%|██████████| 2/2 [00:00<00:00,  2.27it/s]

Updating scores in the original dictionary...
Reranking complete.
Filtering items to rerank (Terminated=False and data_entry exists)...
No items to process.


In [45]:
reranker_model = FlagReranker(check_moedl_path, use_fp16=True)

### Check Additional Execute SQL and Execute

In [53]:
SQL_list_final = []
for index in range(len(dataset)):
    sequence = Check_Model_Data_Sequence[index]['Sequence']
    if sequence==[] or sequence.__contains__('Execute_SQL'):
        SQL_list_final.append(index)
add_sql_list = list(set(SQL_list_final) - set(LLM_query_list['Execute_SQL']['index']))
add_sql_query_list = []
for index in add_sql_list:
    prompt = build_wikitq_prompt_from_df(dataset,wikitq_df_processed[index],index,template_path='../prompts/sql_reason_tabfact.txt',processed=True)
    add_sql_query_list.extend([prompt])
add_sql_response_list = infer_prompts(add_sql_query_list, sample_num=3, temperature=0.7, top_p=0.8,llm_path=llm_path)[0]

Processed prompts: 100%|██████████| 15/15 [00:09<00:00,  1.60it/s, est. speed input=3340.80 toks/s, output=875.38 toks/s]


In [47]:
from tqdm.notebook import tqdm
sample_num = 3
sql_exec_df = {}
valid_parse = 0
warning_list = []
sql_executable_count = []
# exec_sql_index_list = LLM_query_list['Execute_SQL']['index']
# exec_sql_response_list = LLM_query_list['Execute_SQL']['response']
# for index in tqdm(range(len(wikitq_df_processed))):
for i in range(len(add_sql_list)):
    index = add_sql_list[i]
    sql_exec_df[index] = []
    for sample_ind in range(sample_num):
        
        # original_text = eval(wikitq_df_output.iloc[index,-1])[sample_ind]
        original_text = add_sql_response_list[i][sample_ind]
        sql = fix_sql_query(response_text=original_text,table_df=wikitq_df_processed[index],table_title=table_titles[index])
        if [index,sample_ind] in warning_list:
            sql = ''
            print(index,sample_ind)
        if sql!='':
            # print(result)
            try:
                result = executor.sql_exec(sql.replace('``','`'), db, table_id=index,add_row_id = True)
                df = pd.DataFrame(result['rows'],columns=result['header'])
            except:
                df = pd.DataFrame()
        else:
            df = pd.DataFrame()
        sql_exec_df[index].append(df)
        if(len(df)>0):
            valid_parse+=1
            sql_pair = {}
            sql_pair['id'] = index
            sql_pair['sample_ind'] = sample_ind
            sql_pair['sql'] = sql
            sql_pair['table'] = df
            sql_executable_count.append(sql_pair)
sql_exec_df_output_new = merge_clean_and_format_df_dict(sql_exec_df)
    # print(original_text)
for index in sql_exec_df_output_new.keys():
    sql_exec_df_output[index] = sql_exec_df_output_new[index]
processed_table['Execute_SQL'] = sql_exec_df_output

Executing on table_id 1050 ('w1050'): SELECT `title r FROM w WHERE `season = =6 AND ` (SELECT 1 FROM w WHERE `director (s` LIKE '%ilmal% %%)));  AND `season ==)6; `;
Executing on table_id 1050 ('w1050'): SELECT season,no_of_times FROM w   WHERE ififnull("writer (s, "", ") <> "k" G   None;
Executing on table_id 362 ('w362'): None;
Executing on table_id 975 ('w975'): SELECT * FROM w WHERE namename = 'bllich ch r br' AND ?county = ' bltown ';
Executing on table_id 975 ('w975'): SELECT `name FROM w WHERE county = 'bland' AND location LIKE '%bltown%%' ;
Executing on table_id 48 ('w48'): SELECT ipparty, trowmbers FROM ranked_parties WHERE rank IN (r, 2 ``;
Executing on table_id 48 ('w48'): WITH member_counts AS (    SELECT party, SUM(administrativeive) + SUM(cagricultural) + SUM(ccultural and educatinal) + SUM(ustrial and commericial) + SUM(clabbour) + SUM(cnatiionaluniversityofireland) + SUM(cnstitationalyofouDublin) AS total_members FROM w GROUPBY party   ) SELECT c_party FROM tasksWHERE t

/data/workspace/yanmy/HybridRAG/H-STAR/utils/schedule_utils.py:294: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  final_df = unique_df.fillna('')


In [48]:
np.save(f'{tmp_save_path}/Check_Model_Data_Sequence.npy',Check_Model_Data_Sequence)
## step 8: SQP parse aggregate
np.save(f'{tmp_save_path}/processed_table.npy',processed_table)

In [49]:
prompt_list = []
for index in range(len(dataset)):
    sequence = Check_Model_Data_Sequence[index]['Sequence']
    prompt = build_wikitq_prompt_from_df(dataset,Check_Model_Data_Sequence[index]['data_entry']['table'],index,template_path='../prompts/text_reason_tabfact.txt',processed=True)
    if sequence==[] or sequence.__contains__('Execute_SQL'):
        evidence = table_to_str_sql(processed_table['Execute_SQL'][index])
        prompt = prompt + evidence
    prompt_list.append(prompt)
# wikitq_df['instruction'] = prompt_list

In [51]:
## step 11: execute QA
# qa_final = response_vllm(llm,prompt_list, sample_num=1)
qa_final = infer_prompts(prompt_list,temperature=0, top_p=1, sample_num=1,llm_path=llm_path)
wikitq_df = pd.DataFrame(dataset)
wikitq_df['instruction'] = prompt_list
wikitq_df['predict'] = qa_final[0]
## step 12: evaluate
wikitq_df['predict'] = [str(s) for s in qa_final[0]]
acc_all, error_index_all, format_error_index_all = evaluate_predictions(dataset_name, wikitq_df, dataset)

Processed prompts: 100%|██████████| 2024/2024 [01:40<00:00, 20.19it/s, est. speed input=43110.05 toks/s, output=2357.28 toks/s]


  0%|          | 0/2024 [00:00<?, ?it/s]

Correct Samples: 1590; Total Samples: 2024
Accuracy: 78.56


In [31]:
wikitq_df.to_csv(f'{tmp_save_path}/{dataset_name}_{split}_result_30B.csv')

In [62]:
wikitq_df['predict'] = [str(s) for s in qa_final]
acc_all, error_index_all, format_error_index_all = evaluate_predictions(dataset_name, wikitq_df, dataset)

  0%|          | 0/2024 [00:00<?, ?it/s]

Correct Samples: 1848; Total Samples: 2024
Accuracy: 91.30
